### Download Script for PlanetScope Orders

Documentation:
https://github.com/samapriya/porder

https://github.com/tyson-swetnam/porder#order

Anaconda environment: aicore_data
* change user with ´planet init´

In [ ]:
import os
import glob
import pandas as pd
import shutil
import datetime
import warnings; warnings.simplefilter('ignore')
import datetime
import modules.order_utils as util
from pathlib import Path
import zipfile
from tqdm.notebook import tqdm
import numpy as np

### Settings 

In [ ]:
# Set number of days to look into the past for the orders list
DATE_RANGE = 30
#download_dir = 'R:\p_aicore\AICORE\planet_data_inference_grid\downloads'
download_dir = Path(r'Q:\p_aicore_pf\initze\data\planet\planet_data_inference_grid\downloads')

# Intermediate location where to copy downloaded files (faster unzip)
#unzip_dir = Path(r'K:\test\zips')
unzip_dir = Path(r'K:\test\zips')

# Intermediate location where to store the unzipped files
#target_dir = Path(r'K:\test\target')
target_dir = Path(r'K:\test\target')

# Location of final Planet data sorage with RE/PL tiles
tiles_dir = Path(r'Q:\p_aicore_pf\initze\data\planet\planet_data_inference_grid\tiles')

#### Calculate timestamps for orders query 

In [ ]:
today = datetime.datetime.now().strftime('%Y-%m-%d')
start = (datetime.datetime.now() - datetime.timedelta(DATE_RANGE)).strftime('%Y-%m-%d')

#### Show list of running orders 

In [ ]:
running = !porder ostate --state running --start "{start}" --end "{today}"
util.show_orders(running)

#### Show list of successful orders 

In [ ]:
success = !porder ostate --state success --start "{start}" --end "{today}"
df = util.show_orders(success)
df.head(40)

#### Download Order 

Select Order IDs as list or array - see table above

In [ ]:
ORDERS = [0,1,2,3,4] # add order ids here - np.arange(17) for many orders

for order_id in ORDERS:
    util.download_planet_order(df, order_id, download_dir)

## Unzip and reorder files
* add sorting by dl date

In [ ]:
zipfiles = list(Path(download_dir).glob('*/*.zip'))

In [ ]:
COPY = 1
for z in tqdm(zipfiles):
    try:
        if not util.check_zipcontent_exists(z, tiles_dir):
            #print(util.check_zipcontent_exists(z, tiles_dir), z.stem, )
            if COPY:
                print('Copying', z)
                shutil.copy(z, unzip_dir)
    except:
        print(f'Error on {z}')

In [ ]:
#unzipfiles = list(unzip_dir.glob('*/*.zip'))
unzipfiles = list(unzip_dir.glob('*.zip'))

In [ ]:
for zfile in tqdm(unzipfiles[:]):
    try:
        %time util.unzip_PSOrthoTileOrder(zfile, target_dir=target_dir, cleanup_intermediate=False) #needs update for actual processing
    except:
        continue

In [ ]:
dirlist = list(target_dir.glob('*'))

dirlist = [d for d in dirlist if d.is_dir()]
dirlist = [d for d in dirlist if len(d.stem.split('_'))==4]

len(dirlist)

In [ ]:
for d in tqdm(dirlist[:]):
    path_id, tile_id, date, scene_id = d.stem.split('_')
    tile_target = tiles_dir / tile_id / d.stem
    os.makedirs(tiles_dir / tile_id, exist_ok=True)
    if not tile_target.exists():
        shutil.move(str(d), str(tile_target))
    else:
        print(f'{str(tiles_dir / tile_id)} exists')

#### Cleanup Zip Files and extracted data

In [ ]:
for root, dirs, files in os.walk(unzip_dir):
    for f in files:
        os.unlink(os.path.join(root, f))
    for d in dirs:
        shutil.rmtree(os.path.join(root, d))